<a href="https://colab.research.google.com/github/juyoung524/22-1-Spring-/blob/main/20220523.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 3.8 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 77.5 MB/s 
     |████████████████████████████████| 256 kB 95.7 MB/s 
     |████████████████████████████████| 2.0 MB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 50.0 MB/s 
     |████████████████████████████████| 271 kB 54.4 MB/s 
     |████████████████████████████████| 144 kB 61.3 MB/s 
     |████████████████████████████████| 94 kB 934 kB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 11.1 MB 42.0 MB/s 
     |████████████████████████████████| 80 kB 10.6 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 4.0 M

### [Image classification](https://huggingface.co/tasks/image-classification)

큰 제목만큼이 우리가 데모 만드는 제목이라고 이해하면 됨

Image Classification 클릭하면 task별로 설명이 되어 있음 

**시험문제는 저 안에 들어 있는 설명들도 기본적으로 이해하고 있다는 것 가정하고 나올 수 있음 웹페이지 들어가서 이해하기 (개별 설명 이해하기)**

Image classification: 그림으로 들어가서 Output으로 다양한 가능성에 대해 확률과 함께 보여준다. 동영상도 있고 하니까 내부적으로 들어가서 다 확인해 주세요. (기본적인 task에 대한 설명)

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

구체적인 모델 설명, 'How to Use'가 있기도 함.

Transformers 먼저 설치하고 코딩 실행해야 나옴

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.0 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
     |████████████████████████████████| 6.6 MB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

Predicted class: **Egyptian Cat이라고 분류**해 내고 있음

그냥 나오더라. 끝이 아니라, 완벽하게는 아니더라도 대충은 알아야 함

필요한 것들을 import 해왔고, url 보면 우리가 하려고 하는 task가 이미지를 가지고 classify하는 것임. 

내가 upload를 안했다면 코드 내에서 어디선가 이미지가 불러들어져 왔을 것임

url 주소가 제일 끝에 보니까 .jpg가 있어서 '그림이구나' 알 수 있음. 그 사이트로 들어가서 보면 고양이 그림이 나와 있음 (그림에 url만 있는 것임)

jpg의 정보가 숫자의 형태로 들어와 있음

from_pretrained: **pretrained라는 것은 '행렬'임 (AI model)** 데이터로부터 잘 학습되고 만들어진 행렬임. 그걸 'model'이라고 한다

FeatureExtraction할 수 있는 모델이고, extracted된 feature 정보 가지고 classify하는 것임. (**변수 이름 보고 추정 가능**)

흑백사진이 있으면 밝으면 1, 흐리면 0 이런 식으로 숫자 정보로 표현 가능함

컬러면 그런 게 3층의 정보가 있는 것임

그걸 다 쓰기에는 너무나 많은 정보들임.(한 장짜리라도 해상도가 높으면 그냥 숫자들임)

**어떻게 해서든지 압축적으로 정보 뽑아낼 수 있음**

사람에 대한 모든 정보 가지면 거의 무한대의 숫자의 정보가 있는 것임.

이미지 정보로부터 feature 정보 뽑아냄. 뽑아진 feature 정보가 훨씬 적음 (1000개 -> 100개 정도)

압축된 정보를 가지고 'model'이라는 것에 입력으로 들어감. 어떻게 넣었더니 밑에 'idx'라는 게 잡힘. 그것 중에 하나를 predict해내는 것

위에서 뽑아낸 feature 정보로부터 1000개의 이미지들이 준비되어 있음. 그 **1000개에 대해 확률정보가 계산되어 나오는 것**. (1000개의 숫자가 뽑아서 나온다고 생각하면 됨) 1000개의 숫자 다 더하면 1이 됨 (100%니까)

그 중 확률이 제일 큰 것을 찾아라! (predicted_class) - 한 개만 찾으면 됨. 그게 결국 'Egyptian cat'이구나. 라고 이해할 수 있음

In [4]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

필요한 입력 부분을 '입력', 

출력으로 해서 위에 있는 script를 살짝 변형시켜준 것임

그림파일로 들어오면 숫자 형태로 바꿔진. URL로 변형하는 과정에서 아예 assume해도 됨. Gradio로 구현할 때 url 던져주는 것도 아니고 gradio로 변형할 때 그림파일 줄 것임. 그 그림파일은 이미지의 정보로 받아들이면 됨.


In [5]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

0

이거 만들려는 목적이 'demo' 하기 때문에 **떠먹기 좋게 다 만들어주면 좋을 것 같음**. Example, 예시까지 떠먹여주는 부분이 여기임!

url 2개는 class site에 있는 그림 예시들임. 강아지, 호랑이 그림 예시들. 여기에서 url로 받고, 여기 적어주면 그 url들이 됨. 여기 있는 강아지 파일을 이 이름으로 해서, 현재 계정에 class 계정에 있는 두 파일 복사해서 갖다놔라!

여기 두 파일이 있다는 것 가정 하에 examples 적고 file name 적어주면 두 개가 선택하기 쉽게 나타날 것임.

AI, 데이터 영업해야 함. 다른 회사 가서 사장님께 demo 보여줄 때 사장님이 쉽게 클릭하고 볼 수 있도록 (쉽게 클릭할 수 있도록)

예시 부분을 잘 제공하는 것이 Gradio의 강점이니까 유용하게 썼으면 좋겠다.

In [6]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://33022.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f790d5c5950>,
 'http://127.0.0.1:7860/',
 'https://33022.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


입력의 형태, 출력의 형태가 뭔지 유형 정의해야 함

입력으로 이미지 받고, 출력으로 텍스트 뱉을 것이다! 그리고 거기에 맞게 image 넣어주면 되고, 출력으로 text 뱉어주면 됨

위에 있는 함수의 입, 출력 유형을 거기 그냥 적어주면 되는 것임

여기까지만 해주고 launch시키면 실행됨

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

우리가 사용할 구체적인 fill mask 모델 들어와서 그게 다운로드도 많고, 그 모델 선택

How To Use에 나와있는 부분들 중 'sequence'부터는 그렇게 실행하면 저렇게 출력된다는 의미임. 

Mask에 들어갈 말들 예측해서 상위 다섯 개를 보여주고 있음. 위에 있는 demo라기보다는 이렇게 실행하면 밑에 나온 건 결과물이라서 how to use임.

In [7]:
!pip install transformers

In [9]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

이 모델을 들고 온다! 들고와서 그 모델 이름을 'unmasker'라고 하자.

모델을 막 다운로드하고 있음. 이 mask를 지워준다는 의미임.

이 문장을 갖고 실행해 봄 -> 아까 페이지에서 그대로 나오고 있음 (How to use 밑부분에 있던 것)

List로 되어 있음. 결과로 상위 5개가 Dictionary 5개로 나와 있음. Dictionary가 List로 묶여 있는 걸 볼 수 있음.

Demo in Gradio

In [10]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df

Function 먼저 만듦. 모델 불러와서 이름을 'unmasker'로 했고, result 받음.

scores에 보면 list에 아이템이 다섯 개 들어 있음

상위 다섯 개가 list로 dictionary 하나, 두 번째 dictionary... 이렇게 들어 있음

이건 굉장히 쓰기 쉬움. 

**pd.DataFrame이라고 해버리면 그냥 dataframe화가 되어버림.**

List인데 안에 그렇게 들어있는 것을 dataframe해버리면 어떻게 되는지 봅시다.

In [11]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

example 없이 해봅시다.

In [12]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17240.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f79079d0650>,
 'http://127.0.0.1:7861/',
 'https://17240.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


테이터프레임화해서 주니까 훨씬 보기 좋음

예시를 example로 만들어서 주면 훨씬 보기 편리함 (쓰기 좋음)

### [Token classification](https://huggingface.co/tasks/token-classification)

안익숙할 수도 있으니까 정의 먼저 보고 시작하자.

Natural language (NLP)- task 가지고 하는 것임. 그 중에서 하나의 분야임

Natural language의 understanding이라고 하는 것인데, 이게 무슨 말이냐?

어떤 natural language가 주어져 있다면, 이 모든 이야기들이 text가 주어져 있다면 그게 너무 자유도가 높음. 그냥 다 이야기함. 근데 컴퓨터는 모든 가능한 자유도에 대해서 어떻게 하기가 힘듦

이건 뭘 말하는 것이라고 필요한 정보만 뽑아낼 수 있어야 함 (Natural language understanding)

그 중 하나가 Named Entity Recognition (NER) & Part-of-Speech (PoS) tagging

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

어떤 문장이 있을 때 **어떤 명사는 더 정보적으로 salient함 (더 중요함)** 우리가 더 관심 갖고, 집중해야 할 명사들이 있음. 그런 것들을 인식하는 것이 하나의 '기술'임.

모든 명사가 아니라 특정한 것들에 대한 것을 잘 캐치해내는 것임 (AI가)

Location 정보, (LOC) 어떤 조직이다 (ORG), 사람이다 (PER), MISC... 

My name is Wolfgang and I live in Belin.

여기서 'Wolfgang'는 PER

Samsung은 'ORG' (organization)

이런 식으로 뽑아주는 것이 상당히 중요함. 막 씌어진 **natural language에서 중요한 정보를 뽑아주는 것**. NER이 너무너무 중요함. 네이버, 카카오에서 이런 것들을 하기 위한 기술이 고도화되어 있음. (데이터, 모델 계속해서 다듬는 데 인력이 더욱 필요함)

In [13]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


두 개의 pretrained를 불러옴 (AI 모델을 불러오는 것)

AI 모델이 'tokenizer'임.(단어 단위로 쪼개주는 것)

그걸 tokenizer라는 모델로 받아왔고, 이 모델은 그래서 각각의 token들이 어떤 class인지를 classify해주는 것임.

위에 거 잘 보면 pipeline, model, ner의 task를 하는 구나! 인식 가능

Print해봤더니 각각의 token에 대해 설명해 주는 것이 나옴

Demo in Gradio

In [15]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

위에 것과 똑같이 적어주고, 통과해서 받아주는 최종 result를 'dataframe'으로 받아줌

list로 그 안에 dictionary가 여러 개 있을 때 dataframe화해서 던져주니까 되더라

Function 실행해 놓고, example 빼고 실행해 보기

In [16]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

네이버에서는 'natural language processing' 고급인력으로 쳐줌

사용자들이 이거 쓸 때 불편함 느끼지 않도록 하는 것이 굉장히 중요함. 그래서 examples 넣어주는 것 (편리하게 하기 위함)

In [17]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59680.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f790d951850>,
 'http://127.0.0.1:7862/',
 'https://59680.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

단어 속에서 추정할 수 있듯이, 두 주어진 문장이 있을 때 얼마나 유사한지 보는 것임.

Two text가 있을 때 (paragraph/문장 상관 X) 얼마나 similar한지 determine해주는 것

해줘야 하는 게 text를 모든 model들은 input 수준을 그냥 바로 받아들일 수가 없음. 어떤 형태로든지 **컴퓨터가 받아들이는 중간 단계가 필요함** (**feature, vector, embedding**s라는 단어를 사용함)

semantic information- 바꿔줘야지 similarity를 쉽게 계산할 수 있기 때문임.

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [18]:
!pip install sentence_transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=be4f194a07f93e3354331ae80068a830593e40f0b71e550ad86243b812fa1cd7
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


Install 한창 하고, embeddings라는 중요한 개념을 어떻게 이해해야 하는지.

두 개의 비슷한 문장들이 얼마나 비슷한지 계산해 주는 것임

model을 가지고 입력으로 위에 있는 문장을 쓰는구나, 그렇게 했더니 embeddings를 뱉어주는구나!

사실은 불친절하게 나와 있는데 무슨 뜻이냐면, 

몇 개의 숫자가 embedding되어 있는지 확인

sentence transformer 먼저 깔아줘야 함.

In [19]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

[[ 1.88456848e-01  1.74256176e-01  5.44777215e-02  2.90517420e-01
   1.67664081e-01 -4.72069457e-02  6.45580173e-01  1.59808800e-01
   2.26892605e-01 -3.08905598e-02  2.55883545e-01 -5.25876768e-02
  -2.26101175e-01 -5.71063384e-02  1.30426243e-01  1.24953501e-01
   3.17496210e-01  1.94444105e-01 -5.86325705e-01 -1.25861000e-02
   6.09909177e-01  1.64327413e-01  3.33113708e-02 -2.73830682e-01
  -2.89757580e-01 -2.11196810e-01 -2.26138867e-02 -1.70359373e-01
   1.61589950e-01  6.08273745e-02 -2.41624042e-01  1.85792103e-01
   4.27409589e-01  1.92951798e-01 -7.23447800e-02  1.66111022e-01
   1.04428008e-01  2.04772323e-01  2.11167201e-01  1.99740008e-01
  -9.40827355e-02 -1.73836619e-01  6.42733201e-02  2.80254930e-01
  -2.95305759e-01  6.20952919e-02  1.04276791e-01 -2.36439407e-02
   1.29131526e-01 -1.26174763e-01 -1.78990051e-01  3.70057598e-02
  -6.12506211e-01  5.02983071e-02  1.77303478e-01  2.24941239e-01
   1.73860714e-01 -3.84028591e-02 -2.12868378e-01  2.58492470e-01
  -1.21016

print out을 했더니 숫자들이 막 튀어나옴. 

Embeddings에 잘 보면 list가 2개 들어 있음 [[~~~], [~~~]] 이렇게. 

Embeddings 길이 보면 2임. 

384개의 숫자가 있구나. 두 번째도 마찬가지. Embedding for t1, Embedding for t2

In [20]:
len(embeddings)

2

In [21]:
len(embeddings[0])

384

[384개의 숫자] , [384개의 숫자]

embedding for text1, embeddings for t2

이것들이 사실 'feature 벡터'임. 

이 두 개를 가지고 similarity 계산할 수 있음.

예를 들어, 두 개의 벡터가 있을 때 

[1, 2] [2, 1] 이런 벡터가 있다고 쳐봅시다.

**similarity between 2 vectors는 그 벡터들의 각도값의 cos (theta) = cosine similarity** [종이필기 참조]

숫자가 만약에 세 개면 3차원, 네 개면 4차원의 각도 (각도는 하나밖에 없음)

100차원이 된들 원점과 두 벡터는 평면 삼각형을 만들 수밖에 없음 (원점과 평면을 이루고 각도는 이렇게 나오는 것임)

각도가 있으면 cosine 적용해서 -1에서 1 사이의 숫자로 나오는 것임

그걸 계산해 주는 것이 'cosine scores'

embeddings를 두 개로 묶어서 list로 뱉어내는 것까지

그걸 다 합쳐서 function화 시킴

In [22]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

tensor([[0.5726]])

두 벡터가 이루는 값의 cosine값이 나오게 되는 것임. How to use는 embeddings를 2개 list로 묶어서 뱉어주는 것까지 함.

우리는 한 step 더 나아가서 그걸 cosine score 이용해서 계산해주는 것까지 함. 

두 개 다 합쳐서 아래쪽에 function화시킴.

Demo in Gradio

In [23]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

embeddings 나오면 함수를 가져올 수 있음.

return cosine_scores하면 function이 만들어짐.

text1과 text2는 'how to use'에 나오는 것. 두 개를 list화해서 만들어주는 것임. 그러고 나서 아래 쪽에도 마찬가지로 gradio에서 text 유형 두 개로 받아줌. 

In [27]:
examples = [["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]]

In [28]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://50248.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f78fa5f7910>,
 'http://127.0.0.1:7864/',
 'https://50248.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


Similar하지 않음.

"A frog is hopping near the pond"

"I love Korean Food" 두 문장 선택

두 개 실행하면 아니나 다를까 거의 0에 가까운 similarity가 나옴. 0에 가까울수록 similarity는 최소